## 2022 HDAT-DA (Hyundai motor group Data Analyst test) GPSMode 해설

### Hev/PHEV 차량의 에너지 흐름 상태 정보인 HevMode 정보에 대한 상세 데이터 분석 및 이를 통한 에너지 흐름 상태(HevMode) 예측

#### 1. 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import os, random

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything()

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
submission

#### 2. 기초 통계 분석 & EDA

In [ ]:
print(train.columns)

In [ ]:
print(len(train.columns))

In [ ]:
train.describe()

In [ ]:
train['HevMode'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

sns.histplot(train['HevMode'], bins=13)
plt.show()
# 0, 6, 7에 데이터가 쏠려있는 데이터 불균형 문제가 있어 정확한 예측은 불가능할 것으로 판단

In [ ]:
#결측치 분석
temp = train.isna().sum()
temp[temp>0]

변수들간의 상관분석 진행

In [ ]:
# 상관 분석표 확인 (색이 진하면 상관관계 높음)
temp = train.corr()
mask = np.zeros_like(temp)
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(temp,
            cmap = 'RDYlbu_r',
            annot = False,
            mask=mask,
            linewidths=.5,
            cbar_kws={"shrink": .5},
            vmin = -1, vmax=1)
plt.show()

In [ ]:
train[['HL_High']].value_counts() # 대부분의 행에서 같은 값을 가지므로 상관관계 분석이 어려울 것으로 판단되는 변수들

In [ ]:
train[['Inhibit_D']].value_counts()

In [ ]:
train[['Inhibit_N']].value_counts()

In [ ]:
train[['Inhibit_R']].value_counts()

In [ ]:
train[['Inhibit_P']].value_counts()

#### 3. 데이터 전처리

분산 팽창 계수가 10이상인 컬럼 제거 -> 다중공선성 문제를 야기할 수 있는 변수 제거

In [ ]:
from statsmodels.stats.outlier_influence import variance_inflation_factor

drop_columns = ['id']
feature = train.drop(['HevMode'], axis=1)
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(feature.values, i) for i in range(feature.shape[1])]
vif['features'] = features.columns
picked = list(vif[vif['VIF Factor']>10].features)
drop_columns.extend(picked)

train_drop = train.drop(columns=drop_columns)
test_drop = test.drop(columns=drop_columns)

#### 4. 분석 모델 설계 및 예측

Logistic Regression

In [ ]:
train_x, train_y = train_drop.drop('HevMode', axis=1), train_drop['HevMode']

In [ ]:
from sklearn.linear_model import LogisticRegression

linear_model = LogisticRegression()

linear_model.fit(train_x, train_y)

linear_prediction = linear_model.predict(test_drop)

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()

rf_model.fit(train_x, train_y)

rf_prediction = rf_model.predict(test_drop)

#### 5. 제출 파일 생성

In [ ]:
linear_submission = submission.copy()
rf_submission = submission.copy()

linear_submission['HevMode'] = linear_prediction
rf_submission['HevMode'] = rf_prediction

In [ ]:
linear_submission.to_csv('logistic_submission.csv', index=False)
rf_submission.to_csv('rf_submission.csv', index=False)